![LOGO](../../../img/MODIN_ver2_hrz.png)

<center><h2>Scale your pandas workflows by changing one line of code</h2>


# Exercise 3: Not Implemented

**GOAL**: Learn what happens when a function is not yet supported in Modin as well as how to extend Modin's functionality using the DataFrame Algebra.

When functionality has not yet been implemented, we default to pandas

![](../../../img/omnisci_convert_to_pandas.png)

We convert a Modin dataframe to pandas to do the operation, then convert it back once it is finished. These operations will have a high overhead due to the communication involved and will take longer than pandas.

When this is happening, a warning will be given to the user to inform them that this operation will take longer than usual. For example, `DataFrame.mask` is not yet implemented. In this case, when a user tries to use it, they will see this warning:

```
UserWarning: `DataFrame.mask` defaulting to pandas implementation.
```

## Concept for exercise: Default to pandas

In this section of the exercise we will see first-hand how the runtime is affected by operations that are not implemented.

In [ ]:
import modin.pandas as pd
import pandas
import numpy as np
import time
import modin.config as cfg
cfg.StorageFormat.put("omnisci")

frame_data = np.random.randint(0, 100, size=(2**18, 2**8))
df = pd.DataFrame(frame_data)

In [ ]:
pandas_df = pandas.DataFrame(frame_data)

In [ ]:
modin_start = time.time()

print(df.apply(sum))

modin_end = time.time()
print("Modin mask took {} seconds.".format(round(modin_end - modin_start, 4)))

In [ ]:
pandas_start = time.time()

print(pandas_df.apply(sum))

pandas_end = time.time()
print("pandas mask took {} seconds.".format(round(pandas_end - pandas_start, 4)))

## Concept for exercise: Register custom functions

Modin's user-facing API is pandas, but it is possible that we do not yet support your favorite or most-needed functionalities. Your user-defined function may also be able to be executed more efficiently if you pre-define the type of function it is (e.g. map, reduce, etc.) using the DataFrame Algebra. To solve either case, it is possible to register a custom function to be applied to your data.

### Speed improvements
If we were to try and replicate this functionality using the pandas API, we would need to call `df.applymap` with our unary negation function, and subsequently `df.kurtosis` on the result of the first call. Let's see how this compares with our new, custom function!

In [ ]:
start = time.time()

print(pandas_df.applymap(lambda cell_value: ~cell_value).kurtosis())

end = time.time()
pandas_duration = end - start
print("pandas unary negation kurtosis took {} seconds.".format(pandas_duration))

In [ ]:
start = time.time()

print(df.applymap(lambda x: ~x).kurtosis())

end = time.time()
modin_duration = end - start
print("Modin unary negation kurtosis took {} seconds.".format(modin_duration))

In [ ]:
custom_start = time.time()

print(df.neg_kurtosis_custom())

custom_end = time.time()
modin_custom_duration = custom_end - custom_start
print("Modin neg_kurtosis_custom took {} seconds.".format(modin_custom_duration))

In [ ]:
print("As expected, Modin is {}x faster than pandas when chaining the functions; however we see that our custom function is even faster than that - beating pandas by {}x, and Modin (when chaining the functions) by {}x!".format(round(pandas_duration / modin_duration, 2), round(pandas_duration / modin_custom_duration, 2), round(modin_duration / modin_custom_duration, 2)))

## Congratulations! You have just implemented new DataFrame functionality!

## Consider opening a pull request: https://github.com/modin-project/modin/pulls

For a complete list of what is implemented, see the [documentation](https://modin.readthedocs.io/en/latest/UsingPandasonRay/dataframe_supported.html)

## Test your knowledge: Add a custom function for another tree reduce: finding `DataFrame.mad` after squaring all of the values

See the pandas documentation for the correct signature: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mad.html

In [ ]:
modin_mad_custom_start = time.time()

# Implement your function here! Put the result of your custom squared `mad` in the variable `modin_mad_custom`
# Hint: Look at the kurtosis walkthrough above

modin_mad_custom = ...

modin_mad_custom_end = time.time()

In [ ]:
# Evaluation code, do not change!
modin_mad_start = time.time()
modin_mad = df.applymap(lambda x: x**2).mad()
modin_mad_end = time.time()

assert modin_mad_end - modin_mad_start > modin_mad_custom_end - modin_mad_custom_start, \
    "Your implementation was too slow, or you used the chaining functions approach. Try again"

### Now that you are able to create custom functions, you know enough to contribute to Modin!